In [ ]:
!nvidia-smi

Fri Oct 20 11:26:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import tensorflow as tf
print("Tensorflow Version : ", tf.__version__)

Tensorflow Version :  2.14.0


In [2]:
import numpy as np
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img

In [5]:
def get_class_name(folder_names) -> list :
  total_class = []
  for i in folder_names:
    b = i.split('_')
    total_class.append(' '.join(b[4:]))
  return total_class

train_dir = '/content/drive/MyDrive/Project_Sem5/artifacts/data_ingestion/dataset/tomato/train'
test_dir = '/content/drive/MyDrive/Project_Sem5/artifacts/data_ingestion/dataset/tomato/test'
valid_dir = '/content/drive/MyDrive/Project_Sem5/artifacts/data_ingestion/dataset/tomato/valid'

folder_names = glob(train_dir+'/*')
total_class = get_class_name(folder_names)

In [6]:
total_class

[' Tomato Yellow Leaf Curl Virus',
 ' healthy',
 ' Tomato mosaic virus',
 ' Bacterial spot',
 ' Late blight',
 ' Leaf Mold',
 ' Target Spot',
 ' Septoria leaf spot',
 ' Spider mites Two-spotted spider mite',
 ' Early blight']

In [ ]:
# Image size
IMAGE_SIZE = [224, 224]

# downloading the model
base_model = InceptionV3(input_shape= IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in base_model.layers:
  layer.trainable=False

# Flatting the output of the last layer
x = Flatten()(base_model.output)

pred = Dense(len(total_class), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=pred)

model.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical'
  )

Found 4585 images belonging to 10 classes.


In [ ]:
valid_set = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical'
  )

Found 416 images belonging to 10 classes.


In [ ]:
%cd /content/drive/MyDrive/Project_Sem5/Dataset/Tomato

/content/drive/MyDrive/Project_Sem5/Dataset/Tomato


In [ ]:
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath="training/model",
    save_weights_only=False,
    monitor="val_accuracy",
    mode="max",
    verbose=1,
    save_best_only=True
)

In [ ]:
model.fit(
    train_set,
    validation_data=valid_set,
    epochs=30,
    steps_per_epoch=len(train_set),
    validation_steps=len(valid_set),
    callbacks=[checkpoint_callback]
)

Epoch 1/30
287/287 [==============================] - ETA: 0s - loss: 1.7817 - accuracy: 0.8877
Epoch 1: val_accuracy improved from -inf to 0.75481, saving model to training/model
287/287 [==============================] - 100s 349ms/step - loss: 1.7817 - accuracy: 0.8877 - val_loss: 5.7094 - val_accuracy: 0.7548
Epoch 2/30
287/287 [==============================] - ETA: 0s - loss: 1.8713 - accuracy: 0.8857
Epoch 2: val_accuracy improved from 0.75481 to 0.75962, saving model to training/model
287/287 [==============================] - 101s 351ms/step - loss: 1.8713 - accuracy: 0.8857 - val_loss: 6.9153 - val_accuracy: 0.7596
Epoch 3/30
287/287 [==============================] - ETA: 0s - loss: 1.6983 - accuracy: 0.8944
Epoch 3: val_accuracy improved from 0.75962 to 0.80288, saving model to training/model
287/287 [==============================] - 97s 339ms/step - loss: 1.6983 - accuracy: 0.8944 - val_loss: 4.5743 - val_accuracy: 0.8029
Epoch 4/30
287/287 [==============================

In [ ]:
model = keras.models.load_model('./training/model')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

pred = model.predict(test_set)

evaluation = model.evaluate(test_set)

# Print the evaluation metrics
print("Test loss:", evaluation[0])
print("Test accuracy:", evaluation[1])

Found 402 images belonging to 10 classes.
26/26 [==============================] - 4s 80ms/step - loss: 5.3679 - accuracy: 0.8308
Test loss: 5.367871284484863
Test accuracy: 0.8308457732200623
